# Ejercicio 2 - Parcial

Considerar los datasets de la distancia anual entre Urano y Saturno (en unidades astronómicas)
y la generación de energía nuclear por hora en Brasil (kWh), encontrados en este sitio: https:
//tinyurl.com/2k3h8bz4 (los datos están a mitad de página).

### Ayuda

Si se usan funciones de librería scipy.stats, prestar atención a vectorized=False, paired=True

## a)
Encontrar el estimador plug-in del coeficiente de correlación entre ambos conjuntos de datos.

### Desarrollo

Primero empiezo por leer los datos:

In [1]:
import pandas as pd 
import os 

ruta_datos = os.path.join('..', '..', '..', 'datos', 'raw', 'planetary_distance_nuclear_power.csv')

df_distancia_nuclear = pd.read_csv(ruta_datos)

df_distancia_nuclear.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 2 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Planetary distance (AU)                           40 non-null     float64
 1   Nuclear power generation in Brazil (Billion kWh)  40 non-null     float64
dtypes: float64(2)
memory usage: 772.0 bytes


Hallo el estimador plug-in del coeficiente de correlación:

In [2]:
import numpy as np

# Planto la semilla para que los resultados sean reconocibles
np.random.seed(19)

distancia_planetaria = df_distancia_nuclear['Planetary distance (AU)'].values
potencia_nuclear = df_distancia_nuclear['Nuclear power generation in Brazil (Billion kWh)'].values

# Obtengo el estimador plug-in de la correlación
estimador = np.corrcoef(distancia_planetaria, potencia_nuclear)[0, 1]

print(f'Estimador plug-in de la correlación:{estimador}')

Estimador plug-in de la correlación:0.9108140085329718


## b)
Estimar su error estándar usando bootstrap.

### Desarrollo

Calculo el error estándar para este caso usando el método percentil, aunque podría servir con cualquier otro: 

In [8]:
from scipy.stats import bootstrap

# Guardo las observaciones en un array
observaciones = (distancia_planetaria, potencia_nuclear)

# Defino la función que le voy a pasar a bootstrap
def correlacion(grupo1, grupo2):
    return np.corrcoef(grupo1, grupo2)[0, 1]

bs_percentile = bootstrap(observaciones, correlacion, confidence_level= 0.95 ,method= 'percentile', vectorized= False, paired= True)

print(f'Error estándar del estimador del coeficiente de correlación con bootstrap: {bs_percentile.standard_error}')


Error estándar del estimador del coeficiente de correlación con bootstrap: 0.01947442620811697


## c)
Encontrar un intervalo de confianza de nivel 0.95 usando los métodos normal, percentile
bootstrap, basic bootstrap y BCa.

### Desarrollo

Para realizar este inciso utilizo la función bootstrap de la librería scipy como antes hice para calcular el error estándar y además utilizo las funciones y datos definidos anteriormente:

In [12]:
intervalo_percentil = bootstrap(observaciones, correlacion, confidence_level= 0.95 ,method= 'percentile', vectorized= False, paired= True)
intervalo_basic = bootstrap(observaciones, correlacion, confidence_level= 0.95 ,method= 'basic', vectorized= False, paired= True)
intervalo_bca = bootstrap(observaciones, correlacion, confidence_level= 0.95 ,method= 'bca', vectorized= False, paired= True)
intervalo_normal = [estimador -1.96*intervalo_percentil.standard_error, estimador +1.96*intervalo_percentil.standard_error]

print(f'Intervalo de confianza para la desviación estándar usando el método normal: [{intervalo_normal[0]}, {intervalo_normal[1]}]')
print(f'Intervalo de confianza para la desviación estándar usando el método percentil: [{intervalo_percentil.confidence_interval.low}, {intervalo_percentil.confidence_interval.high}]')
print(f'Intervalo de confianza para la desviación estándar usando el basic bootstrap: [{intervalo_basic.confidence_interval.low}, {intervalo_basic.confidence_interval.high}]')
print(f'Intervalo de confianza para la desviación estándar usando el BCa bootstrap: [{intervalo_bca.confidence_interval.low}, {intervalo_bca.confidence_interval.high}]')

Intervalo de confianza para la desviación estándar usando el método normal: [0.8724550959247062, 0.9491729211412374]
Intervalo de confianza para la desviación estándar usando el método percentil: [0.8683198605926068, 0.9449579303586303]
Intervalo de confianza para la desviación estándar usando el basic bootstrap: [0.8773016159260784, 0.953217341200946]
Intervalo de confianza para la desviación estándar usando el BCa bootstrap: [0.8607590687160194, 0.941488991164435]
